In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [2]:
# TAG Word
PAD = "<PADDING>" 
STA = "<START>"  
END = "<END>" 
OOV = "<OOV>"

# Tag Index
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# Data Type
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# MAXiMUM Sequnce
max_sequences = 30

# Embedding Vector Dimension
embedding_dim = 100

# LSTM Hidden Layer Dimension
lstm_hidden_dim = 128

# Filter
RE_FILTER = re.compile("[.,!?\"':;~()]")

# Chatbot Data Load
chatbot_data = pd.read_csv('smuChattinghaejoTraining.txt', names=['Q', 'A'], sep='\t')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [3]:
# Question Data Size = Answer Data Size
len(question)


21245

In [4]:
# 랜덤한 챗봇 데이터 출력 (10개)
ramdomNum = np.random.randint(0, len(question), size=10)
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 윤종범 교수님 위치를 알 수 있을까?
A : professor 위치

Q : 문헌정보학전공 사무실 위치가 어디야?
A : office 문헌정보학전공 위치

Q : 스포츠건강관리과 어떻게 가나염
A : office 스포츠건강관리전공 위치

Q : 융경과 위치가 어디쯤이야
A : office 융합경영학과 위치

Q : 경영학과 번호가 어떻게 되나요
A : office 경영학부 연락처

Q : 가복 과사무실 위치좀
A : office 가족복지학과 위치

Q : 보건 연락처좀 알려주세요
A : office 보건건강관리센터 연락처

Q : 컴과 과사 번호가 뭐야?
A : office 컴퓨터과학과 연락처

Q : 조예 과사 번호좀
A : office 조형예술전공 연락처

Q : 최종술 교수님 사무실 번호가 뭐야
A : professor 연락처



In [5]:
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [6]:
# 형태소분석 수행
question = pos_tag(question)
# Question 만 형태소 분석 실시 (Answer는 이미 정해진 타입이 있기 때문)

# 변경된 점 출력
for i in ramdomNum:
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 윤 종범 교수 님 위치 를 알 수 있을까
A : professor 위치

Q : 문헌정보학 전공 사무실 위치 가 어디 야
A : office 문헌정보학전공 위치

Q : 스포츠 건강 관리 과 어떻게 가나 염
A : office 스포츠건강관리전공 위치

Q : 융경 과 위치 가 어디 쯤 이야
A : office 융합경영학과 위치

Q : 경영학 과 번호 가 어떻게 되나요
A : office 경영학부 연락처

Q : 가복 과 사무실 위치 좀
A : office 가족복지학과 위치

Q : 보건 연락처 좀 알려주세요
A : office 보건건강관리센터 연락처

Q : 컴 과 과사 번호 가 뭐 야
A : office 컴퓨터과학과 연락처

Q : 조예 과사 번호 좀
A : office 조형예술전공 연락처

Q : 최종 술 교수 님 사무실 번호 가 뭐 야
A : professor 연락처



In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words = sorted(list(words))
words[:0] = [PAD, STA, END, OOV]



In [8]:
# 단어 개수
len(words)


601

In [9]:
# 단어 출력
words


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '1',
 '10월',
 '10일',
 '11월',
 '11일',
 '12월',
 '1월',
 '1학기',
 '2',
 '2월',
 '2일',
 '2학기',
 '3월',
 '3일',
 '4월',
 '4일',
 '5월',
 '6월',
 '6일',
 '?',
 'HELLO',
 'HI',
 'Hello',
 'Hi',
 'IOT',
 'cafeteria',
 'date',
 'event',
 'hi',
 'iot',
 'now',
 'office',
 'professor',
 'ㅇ',
 'ㅇㅅㅇ',
 'ㅎㅇ',
 '가',
 '가나',
 '가는',
 '가복',
 '가야해',
 '가요',
 '가족',
 '가족복지학과',
 '갈',
 '감',
 '강',
 '강성',
 '강의',
 '강의평가',
 '개강',
 '개교',
 '개교기념일',
 '개발',
 '거',
 '건강',
 '게임',
 '게임전공',
 '게임학',
 '겜',
 '겨울',
 '경',
 '경과',
 '경영',
 '경영학',
 '경영학부',
 '경제',
 '경제금융학부',
 '경제학',
 '계',
 '계셔',
 '계신지',
 '계심',
 '계절',
 '계절수업',
 '고',
 '고사',
 '공',
 '공간',
 '공간환경학부',
 '공과',
 '공임',
 '공학',
 '공학교육혁신센터',
 '공환',
 '과',
 '과사',
 '관리',
 '관리팀',
 '관하여',
 '교수',
 '교원',
 '교원인사',
 '교원인사팀',
 '교육',
 '교육학',
 '교육학과',
 '교직',
 '교직지원센터',
 '교차',
 '교학',
 '교학팀',
 '국',
 '국가',
 '국가안보학과',
 '국교',
 '국안',
 '국어',
 '국어교육',
 '국어교육과',
 '국제',
 '국제언어문화교육원',
 '국제학생지원팀',
 '궁금한',
 '권',
 '귀',
 '귀지',
 '글',
 '글경',
 '글로벌',
 '글로벌경영학과',
 '글로벌랭귀지센

In [10]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [11]:
np.save("word_to_index",word_to_index)
np.save("index_to_word",index_to_word)

In [12]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items()))


{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '1': 4,
 '10월': 5,
 '10일': 6,
 '11월': 7,
 '11일': 8,
 '12월': 9,
 '1월': 10,
 '1학기': 11,
 '2': 12,
 '2월': 13,
 '2일': 14,
 '2학기': 15,
 '3월': 16,
 '3일': 17,
 '4월': 18,
 '4일': 19,
 '5월': 20,
 '6월': 21,
 '6일': 22,
 '?': 23,
 'HELLO': 24,
 'HI': 25,
 'Hello': 26,
 'Hi': 27,
 'IOT': 28,
 'cafeteria': 29,
 'date': 30,
 'event': 31,
 'hi': 32,
 'iot': 33,
 'now': 34,
 'office': 35,
 'professor': 36,
 'ㅇ': 37,
 'ㅇㅅㅇ': 38,
 'ㅎㅇ': 39,
 '가': 40,
 '가나': 41,
 '가는': 42,
 '가복': 43,
 '가야해': 44,
 '가요': 45,
 '가족': 46,
 '가족복지학과': 47,
 '갈': 48,
 '감': 49,
 '강': 50,
 '강성': 51,
 '강의': 52,
 '강의평가': 53,
 '개강': 54,
 '개교': 55,
 '개교기념일': 56,
 '개발': 57,
 '거': 58,
 '건강': 59,
 '게임': 60,
 '게임전공': 61,
 '게임학': 62,
 '겜': 63,
 '겨울': 64,
 '경': 65,
 '경과': 66,
 '경영': 67,
 '경영학': 68,
 '경영학부': 69,
 '경제': 70,
 '경제금융학부': 71,
 '경제학': 72,
 '계': 73,
 '계셔': 74,
 '계신지': 75,
 '계심': 76,
 '계절': 77,
 '계절수업': 78,
 '고': 79,
 '고사': 80,
 '공': 81,
 '공간': 82,
 '공간환경학부': 83,
 '공과': 84,
 '공

In [13]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items()))


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '1',
 5: '10월',
 6: '10일',
 7: '11월',
 8: '11일',
 9: '12월',
 10: '1월',
 11: '1학기',
 12: '2',
 13: '2월',
 14: '2일',
 15: '2학기',
 16: '3월',
 17: '3일',
 18: '4월',
 19: '4일',
 20: '5월',
 21: '6월',
 22: '6일',
 23: '?',
 24: 'HELLO',
 25: 'HI',
 26: 'Hello',
 27: 'Hi',
 28: 'IOT',
 29: 'cafeteria',
 30: 'date',
 31: 'event',
 32: 'hi',
 33: 'iot',
 34: 'now',
 35: 'office',
 36: 'professor',
 37: 'ㅇ',
 38: 'ㅇㅅㅇ',
 39: 'ㅎㅇ',
 40: '가',
 41: '가나',
 42: '가는',
 43: '가복',
 44: '가야해',
 45: '가요',
 46: '가족',
 47: '가족복지학과',
 48: '갈',
 49: '감',
 50: '강',
 51: '강성',
 52: '강의',
 53: '강의평가',
 54: '개강',
 55: '개교',
 56: '개교기념일',
 57: '개발',
 58: '거',
 59: '건강',
 60: '게임',
 61: '게임전공',
 62: '게임학',
 63: '겜',
 64: '겨울',
 65: '경',
 66: '경과',
 67: '경영',
 68: '경영학',
 69: '경영학부',
 70: '경제',
 71: '경제금융학부',
 72: '경제학',
 73: '계',
 74: '계셔',
 75: '계신지',
 76: '계심',
 77: '계절',
 78: '계절수업',
 79: '고',
 80: '고사',
 81: '공',
 82: '공간',
 83: '공간환경학부',
 84: '공과',
 85

In [14]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [15]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)


In [16]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

In [17]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)


In [18]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [19]:
# ---------- MARK:- Model Encoder

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



# ---------- MARK:- Model Decoder

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# ---------- MARK:- Model Setting

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

In [20]:
# ---------- MARK:- Model Encoder

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



# ---------- MARK:- Predict Model Decoder

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [21]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

# Model.fit

In [22]:
# 모델 교육
model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=50,
                        batch_size=64,
                        verbose=1)
# 나중에 챗봇으로 쓸 때 epoch랑 batch_size 수치 잘 수정하기
# 추후 빡세게 Model.fit 하고 Model 저장 후 Load해서 하기

Epoch 1/50
332/332 [==============================] - 66s 106ms/step - loss: 0.8358 - acc: 0.8874
Epoch 2/50
332/332 [==============================] - 34s 103ms/step - loss: 0.1512 - acc: 0.9568
Epoch 3/50
332/332 [==============================] - 33s 99ms/step - loss: 0.1054 - acc: 0.9705
Epoch 4/50
332/332 [==============================] - 34s 102ms/step - loss: 0.0814 - acc: 0.9752
Epoch 5/50
332/332 [==============================] - 32s 97ms/step - loss: 0.0612 - acc: 0.9810
Epoch 6/50
332/332 [==============================] - 32s 98ms/step - loss: 0.0444 - acc: 0.9866
Epoch 7/50
332/332 [==============================] - 32s 97ms/step - loss: 0.0301 - acc: 0.9914
Epoch 8/50
332/332 [==============================] - 34s 103ms/step - loss: 0.0205 - acc: 0.9940
Epoch 9/50
332/332 [==============================] - 35s 105ms/step - loss: 0.0150 - acc: 0.9957
Epoch 10/50
332/332 [==============================] - 35s 105ms/step - loss: 0.0103 - acc: 0.9971
Epoch 11/50
332/332 [==

In [23]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [24]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)
    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    
    print(indexs)
    return sentence

In [25]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 번호가 뭐야')
input_seq


array([[ 94, 158, 230,  40, 208, 336,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [26]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[36, 361, 2]


'professor 연락처 '

In [27]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교수님 위치가 뭐야?')
input_seq


array([[ 94, 158, 385,  40, 208, 336,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [28]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[36, 385, 2]


'professor 위치 '

In [29]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('컴퓨터과학과 위치가 어디야')
input_seq


array([[525,  89, 385,  40, 338, 336,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [30]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


[35, 526, 385, 2]


'office 컴퓨터과학과 위치 '

In [31]:
generate_text(make_predict_input(input()))

gd
[320, 504, 505, 425, 553, 463, 564, 93, 117, 461, 434, 23, 2]


'안녕하세요. 채팅해조 챗봇 입니다. 학사 정보, 학식에 관하여 궁금한 점이 있으신가요 ? '

#### Kakao-brain의 Pororo 사용 : Text에서 PERSON을 찾을 수 있음 -> Professor 명령일 경우 PERSON을 추출하여 Professor 명령에 PERSON을 넣어줌
from pororo import Pororo
ner = Pororo(task="ner",lang='ko')


sentence = input()
pf = 'professor' in generate_text(make_predict_input(sentence))
if pf:
    for name in ner(sentence):
        if 'PERSON' in name:
            orderText = generate_text(make_predict_input(sentence)).split(" ")
            print(orderText[0] + " " +name[0]+ " "+orderText[1] )
else:
    print(generate_text(make_predict_input(sentence)))
    
#### EX) 홍길동 교수님 연락처 알려줘 -> professor 연락처 + 홍길동 -> professor 홍길동 연락처

# Model Save

In [32]:
from keras.models import load_model

model.save('chatbot_model.h5')

# Model Load

from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('encoder_model.json', 'encoder_model_weights.h5')
decoder = load_model('decoder_model.json', 'decoder_model_weights.h5')

from keras.models import load_model
model = load_model('chatbot_model.h5')

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    60100       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    60100       input_2[0][0]                    
______________________________________________________________________________________________